In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
file = pd.read_csv('discount.csv', index_col = 0, parse_dates = True)
df = file['discount']
df

term
0.25     0.994018
0.50     0.991847
0.75     0.985719
1.00     0.979189
1.25     0.972400
1.50     0.965407
1.75     0.958313
2.00     0.946648
3.00     0.918169
4.00     0.890856
5.00     0.864410
6.00     0.838709
7.00     0.813701
8.00     0.789175
9.00     0.765001
10.00    0.741630
11.00    0.718651
12.00    0.696587
15.00    0.634569
20.00    0.544388
25.00    0.470307
30.00    0.408581
40.00    0.311021
50.00    0.241662
Name: discount, dtype: float64

## G2++ Model

In [3]:
class Model:
    def __init__(self,x):
        self.x0 = x[0]
        self.y0 = x[1]
        self.a = x[2]             
        self.sig = x[3]
        self.b = x[4]
        self.eta = x[5]
        self.rho = x[6]

    def B(self,z,tau):
        return (1-np.exp(-z*(tau)))/z

    def V(self,tau):
        tmp1 = self.sig**2/(self.a**2)*(tau + 2/self.a*np.exp(-self.a*tau) - 1/(2*self.a)*np.exp(-2*self.a*tau) - 3/(2*self.a))
        tmp2 = self.eta**2/(self.b**2)*(tau + 2/self.b*np.exp(-self.b*tau) - 1/(2*self.b)*np.exp(-2*self.b*tau) - 3/(2*self.b))
        tmp3 = 2*self.rho*self.sig*self.eta/(self.a*self.b)*(tau + 1/self.a*(np.exp(-self.a*tau)-1) + 1/self.b*(np.exp(-self.b*tau)-1)
                                                            - 1/(self.a+self.b)*(np.exp(-(self.a+self.b)*tau)-1))
        return tmp1+tmp2+tmp3
        
    def bond_price(self,t,T,P_t2,P_t1):
        A = P_t2/P_t1*np.exp((self.V(T-t)-self.V(T)+self.V(t))/2)
        price = A*np.exp(-self.B(a,T-t)*self.x0 - self.B(b,T-t)*self.y0)
        return price
    
    def update_params(self,x):
        self.x0 = x[0]
        self.y0 = x[1]
        self.a = x[2]             
        self.sig = x[3]
        self.b = x[4]
        self.eta = x[5]
        self.rho = x[6]

In [4]:
x0 = 1.0/100
y0 = .705/100 # x0+y0=1.705, overnight libor rate on May 12 2018 is 1.705
a = 5
sig = .15
b = .35
eta = .08
rho = -0.9

m = Model([x0,y0,a,sig,b,eta,rho])

In [5]:
# test if bond price can work well
m.bond_price(0,40,0.311021,1)

0.30420979959478234

## Calibration

In [6]:
from scipy.optimize import least_squares

class Fitter:
    def __init__(self,model,bond_data,term):
        self.model = model
        self.data = bond_data
        self.term = term
        
    def day_residual(self,x):
        self.model.update_params(x)
        mode_price = [self.model.bond_price(0,self.term[i],self.data[i],1) for i in range(len(self.term))]
        return mode_price - self.data
    
    def fit(self,x_guess,solver='lm'):
        res = least_squares(self.day_residual,x_guess, method=solver)#obj function,x0 guess
        return res

In [7]:
term = df.index.values
bond = Fitter(m,df.values,term)
print("the calibrated parameters are wield. x0,y0,a,sigma,b,eta,rho: ")
bond.fit([1.0/100, 0.705/100, 4, 0.09, 0.1, 0.08, -0.6]).x

the calibrated parameters are wield. x0,y0,a,sigma,b,eta,rho: 


array([  2.78994781e-08,  -3.51890548e-09,   4.00000000e+00,
         9.00000000e-02,   1.00000000e-01,   8.00000000e-02,
        -6.00000000e-01])

## computed bond option prices based on analytical formula

In [8]:
# as parameters are not reasonable, I just estimate.
x0 = 0.01
y0 = 0.00705
a = 5
b = 0.35
sigma = 0.15
rho = -0.9
eta = 0.08
# P_t_T = df[7]
# P_t_S = df[8]
P_t_T = 0.946648
P_t_S = 0.918169
K = .96

In [9]:
from scipy.stats import norm
import math
tmp1 = sigma**2/(2*a**3)*((1-np.exp(-a*1))**2)*(1-np.exp(-2*a*2))
tmp2 = eta**2/(2*b**3)*((1-np.exp(-b*1))**2)*(1-np.exp(-2*b*2))
tmp3 = 2*rho*sigma*eta/(a*b*(a+b))*(1-np.exp(-a*1))*(1-np.exp(-b*1))*(1-np.exp(-(a+b)*2))

In [12]:
cov = math.sqrt(tmp1+tmp2+tmp3)
# call option with T written on zero-coupon bond with S computed by analytical formula
ZBC = P_t_S* norm.cdf(np.log(P_t_S/(K*P_t_T))/cov + .5*cov) - P_t_T*K*norm.cdf(np.log(P_t_S/(K*P_t_T))/cov - .5*cov)
print("bond call option based on analytical formula: ", 100*ZBC)
print("Error between analytical price and simulated price",(100*ZBC-2.877804)/(100*ZBC))

bond call option based on analytical formula:  2.89223547281
Error between analytical price and simulated price 0.00498972955346


In [11]:
# put
ZBP = -P_t_S* norm.cdf(np.log(P_t_S/(K*P_t_T))/cov - .5*cov) + P_t_T*K*norm.cdf(np.log(P_t_S/(K*P_t_T))/cov + .5*cov) 
ZBP

0.018369280106472985